# eFarmersHub Data Analysis
eFarmersHub data are stored in `gds_database` for data visualization. There are 8 tables:
1. Income generating tables: Sale, Machine Rent & Advisory
2. Expenditure tables: Purchase, Processing, Expense
3. User table: This table stores all the user data

The script utilizes `SQLAlchemy` as a database toolkit for CRUD operation while `Pandas` is used for data manipulation.

In [1]:
# Import Modules
# data manipulation and analysis
import pandas as pd
import numpy as np

# database toolkit
from sqlalchemy import create_engine, MetaData, inspect, Table, Column, Integer, String, Date, Numeric
from sqlalchemy.engine.url import URL
from sqlalchemy.sql import select

# read env file
from dotenv import load_dotenv
import os

# path handling
from pathlib import Path

# logging
import logging

In [ ]:
# load env variables
dotenv_path = Path("./.env")
load_dotenv(dotenv_path=dotenv_path)

USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")

print(DATABASE)